# Predicting Price Of Renting An Apartment In Lagos State.

The objective of this project is predicting the cost of renting an apartment in lagos state based on the area, number of bed rooms and number of toilets.
Data set for this project was gathered from https://tolet.ng, also scrapper for gathered the data can be found on my github here -> https://github.com/victorlenerd/toletngscrapper.

## Loading and cleaning data.

In [17]:
## Loading Data

import pandas as pd

data = pd.read_json('./data.json')

In [9]:
data.tail()

,area,no_bath,no_bed,no_toilets,price
9041,"Mafoluku road, Isolo, Mafoluku Oshodi Lagos",1,1,1,280000.0
9042,"Mafoluku road, Mafoluku Oshodi Lagos",1,1,1,280000.0
9043,,0,0,0,NaN
9044,,0,0,0,NaN
9045,,0,0,0,NaN


In [6]:
data.describe()

,no_bath,no_bed,no_toilets,price
count,9046.000000,9046.000000,9046.000000,7.846000e+03
mean,2.125138,2.170462,2.404378,3.302896e+06
std,1.588876,1.581791,1.889390,4.822354e+07
min,0.000000,0.000000,0.000000,1.000000e-02
25%,1.000000,1.000000,1.000000,5.000000e+05
50%,2.000000,2.000000,3.000000,1.000000e+06
75%,3.000000,3.000000,4.000000,2.875000e+06
max,10.000000,10.000000,10.000000,4.000000e+09


In [15]:
# The scrapper gathered some wrong values from dom elements that had the same property. It's best to drop them
data = data.dropna()

In [20]:
data.describe()

,no_bath,no_bed,no_toilets,price
count,9046.000000,9046.000000,9046.000000,7.846000e+03
mean,2.125138,2.170462,2.404378,3.302896e+06
std,1.588876,1.581791,1.889390,4.822354e+07
min,0.000000,0.000000,0.000000,1.000000e-02
25%,1.000000,1.000000,1.000000,5.000000e+05
50%,2.000000,2.000000,3.000000,1.000000e+06
75%,3.000000,3.000000,4.000000,2.875000e+06
max,10.000000,10.000000,10.000000,4.000000e+09


In [14]:
data.tail()

,area,no_bath,no_bed,no_toilets,price
9038,- Abule Egba Abule Egba Lagos,2,2,3,500000.0
9039,- Lekki Phase 1 Lekki Lagos,5,5,6,5500000.0
9040,"Awuse Estate, Opebi, Lagos Opebi Ikeja Lagos",5,5,6,2500000.0
9041,"Mafoluku road, Isolo, Mafoluku Oshodi Lagos",1,1,1,280000.0
9042,"Mafoluku road, Mafoluku Oshodi Lagos",1,1,1,280000.0


In [27]:
from collections import Counter
area_counts = Counter(data['area'])
area_counts

Counter({'HI-TECH BY L.B.S Off Lekki Epe Expressway Ajah Lagos': 1,
         'Igando/Ikotun, Lagos Ikotun Igando Lagos': 17,
         ' 39 Alfred Rewane Road Ikoyi Ikoyi S.w Ikoyi Lagos': 5,
         'Water Corporation Road Victoria Island Extension Victoria Island Lagos': 5,
         'Karimu Kotun Street Lagos Island Lagos': 5,
         'Agungi/Idado Axis Lekki Agungi Lekki Lagos': 1,
         'Ayobo/Ipaja, Lagos Ipaja Lagos': 12,
         'Ligali Ligali Ayorinde Victoria Island Lagos': 1,
         '- Ahmadu Bello Way Victoria Island Lagos': 1,
         'Lakowe - Lakowe Ajah Lagos': 1,
         '- Thomas Estate Ajah Lagos': 9,
         'Palace Road Oniru Victoria Island Extension Victoria Island Lagos': 1,
         '- Oniru Victoria Island Extension Victoria Island Lagos': 18,
         'Ipaja, Alimosho, Lagos Ipaja Lagos': 16,
         'Off Fola agoro akoka Akoka Yaba Lagos': 1,
         'Festiva Road, Victoria Island Extension Victoria Island Lagos': 1,
         '': 1200,
         '-

### Using areas.json to make area property more structured.

In [30]:
## Loading Areas
areas = pd.read_json('./areas.json')

In [31]:
areas

,0
0,Abule Egba
1,Agege
2,Ajah
3,Ajaokuta
4,Alimosho
5,Amuwo Odofin
6,Apapa
7,Arepo
8,Badagry
9,Egbe Idimu
